In [ ]:
# Install mysql module
!pip3.9 install mysql-connector-python

In [ ]:
# Make sure ur IP is on file
!curl ipecho.net/plain

In [ ]:
# Connect to server
import mysql.connector
import random
import pandas as pd

mydb = mysql.connector.connect(
    host="35.232.56.109",
    user="root",
    password="ah08EbAGxF04I503",
    database="SpotifyHelpr"
)

mycursor = mydb.cursor()

In [ ]:
# Load CSV
df = pd.read_csv("assets/SpotifyFeatures.csv")
ids = df.track_id.tolist()

# Get 1000 most popular songs
df2 = df.drop_duplicates(subset=["track_id"]).sort_values("popularity",ascending=False)
df2head = df2.head(1000)

In [ ]:
# Print results from query
def print_cursor(cursor):
    myresult = cursor.fetchall()
    for x in myresult:
        print(x)

In [ ]:
# Commit changes to server
def commit():
    mydb.commit()

In [ ]:
# Create Users table
mycursor.execute("""CREATE TABLE Users (
	userId   INT         NOT NULL,
	userName VARCHAR(50),
	password VARCHAR(50),
	apiToken VARCHAR(50),

	PRIMARY KEY(userId)
);""")

In [ ]:
# Populate Users table
for i in range(1009):
    mycursor.execute(f'Insert into Users Values({i+1}, "test{i+1}", "pwd", "api{i+1}");')

In [ ]:
# Create the Songs table
mycursor.execute("""CREATE TABLE Songs (
	songId         VARCHAR(50) NOT NULL,
	name           VARCHAR(150),
	artist         VARCHAR(50),
	genre          VARCHAR(50),
	url            VARCHAR(50),
	likenessFactor REAL,

	PRIMARY KEY(songId)
);""")

In [ ]:
# Populate Songs table
# idx = -1
l = []
for i, entry in df2head.iterrows():
    # idx += 1
    # if idx < 134:
    #     continue
    entry = list(entry)
    url = f"open.spotify.com/track/{entry[3]}"
    entry2 = entry[2].replace('"', '')
    mycursor.execute(f'Insert into Songs Values("{entry[3]}", "{entry2}", "{entry[1]}", "{entry[0]}", "{url}", {random.randint(1, 20)});')
    commit()

In [ ]:
# Create the Reviews table
mycursor.execute("""CREATE TABLE Reviews (
	reviewId INT          NOT NULL,
	rating   REAL,
	body     VARCHAR(140),

	songId   VARCHAR(50),
	userId   INT,

	PRIMARY KEY(reviewId),
	FOREIGN KEY(songId) REFERENCES Songs(songId) ON DELETE CASCADE,
	FOREIGN KEY(userId) REFERENCES Users(userId) ON DELETE CASCADE
);""")

In [ ]:
# Populate Reviews table
l=[]
d = {}
idx = 0
for i, entry in df2head.iterrows():
    d[idx] = entry[3]
    idx += 1

for i in range(1000):
    f = [d[random.randint(0, 999)], random.randint(0, 1000)]
    while(f in l):
        f = [d[random.randint(0, 999)], random.randint(0, 1000)]

    l.append(f)

    mycursor.execute(f'insert into Reviews values({i}, {random.randint(0, 10)}, "body{i}", "{f[0]}", {f[1]});')
    commit()

In [ ]:
# Create Playlists table
mycursor.execute("""CREATE TABLE Playlists (
	playlistId   INT         NOT NULL,
	playlistName VARCHAR(50),

	userId       INT,

	PRIMARY KEY(playlistId),
	FOREIGN KEY(userId) REFERENCES Users(userId) ON DELETE CASCADE
);""")

In [ ]:
# Populate Playlists table
for i in range(1000):
    mycursor.execute(f'Insert into Playlists Values({i}, "playlist{i}", {random.randint(0, 1000)});')
    commit()

In [ ]:
# Create SongsFoundIn table
mycursor.execute("""CREATE TABLE SongsFoundIn (
	songId     VARCHAR(50) NOT NULL,
	playlistId INT         NOT NULL,

	PRIMARY KEY(songId, playlistId),
	FOREIGN KEY(songId)     REFERENCES Songs(songId)         ON DELETE CASCADE,
	FOREIGN KEY(playlistId) REFERENCES Playlists(playlistId) ON DELETE CASCADE
);""")

In [ ]:
# Populate SongsFoundIn table
d = {}
idx = 0
for i, entry in df2head.iterrows():
    d[idx] = entry[3]
    idx += 1

for i in range(1000):
    l = []
    for j in range(random.randint(3, 8)):
        f = d[random.randint(0, 999)]
        while(f in l):
            f = d[random.randint(0, 999)]

        l.append(f)
        mycursor.execute(f'Insert into SongsFoundIn Values("{f}", {i});')
        commit()
    

In [ ]:
# Create table Friends
mycursor.execute("""CREATE TABLE Friends (
	friendId INT  NOT NULL,
	userId   INT  NOT NULL,
	fScore   REAL,

	PRIMARY KEY(friendId, userId),
	FOREIGN KEY(friendId) REFERENCES Users(userId) ON DELETE CASCADE,
	FOREIGN KEY(userId)   REFERENCES Users(userId) ON DELETE CASCADE
);""")

In [ ]:
# Create random friendships

friendships = []
for i in range(2000):
    f1 = random.randint(0, 999)
    f2 = random.randint(0, 999)
    while(f1 == f2):
        f2 = random.randint(0, 999)

    l = [f1, f2]
    l2 = [f2, f1]

    if l not in friendships and l2 not in friendships:
        mycursor.execute(f'Insert into Friends Values({l[1]}, {l[0]}, {random.randint(1, 10)});')
        friendships.append(l)
        

In [ ]:
query = """SELECT name, artist, r.avg_rating
FROM Songs
NATURAL JOIN (SELECT songId FROM SongsFoundIn WHERE playlistId = 700) s
LEFT JOIN (SELECT songId, avg(rating) AS avg_rating FROM Reviews GROUP BY songId) r
ON Songs.songId = r.songId;"""

mycursor.execute(query) # Find the name, artist, and average ratings for all the songs in a given playlist (eg. playlistId 700)
print_cursor(mycursor)

In [ ]:
query = """SELECT Users.userName, sub.name, sub.artist, sub.body, sub.rating
FROM 
(SELECT Reviews.userId, Songs.name, Songs.artist, Reviews.body, Reviews.rating
	FROM Songs NATURAL JOIN Reviews) sub
JOIN Users ON Users.userId = sub.UserId
WHERE sub.userId IN (SELECT friendId FROM Friends WHERE userId = 90) 

UNION

SELECT Users.userName, sub.name, sub.artist, sub.body, sub.rating
FROM
(SELECT Reviews.userId, Songs.name, Songs.artist, Reviews.body, Reviews.rating
	FROM Songs NATURAL JOIN Reviews) sub 
JOIN Users ON Users.userId = sub.UserId
WHERE sub.userId IN (SELECT userId FROM Friends WHERE friendId = 90);"""

mycursor.execute(query) # Find the username, song name, artist, body, and ratings for all the reviews written by all of a User's friends (eg. userId = 90). We are using a UNION in order to find occurrences where the user is a friend of another user (as our friends relations are directed)
print_cursor(mycursor)